In [25]:
import pandas as pd
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import json

In [30]:
Content-Type: application/json; charset=utf-8

SyntaxError: illegal target for annotation (<ipython-input-30-4c1f71d405c8>, line 1)

In [35]:
data = pd.read_csv('/Users/rimshad/Downloads/scholar-vertical-search-engine-rimshad/scholar-vertical-search-engine-rimshad/Classification/imdb.csv')
dataCopy = data
data.head()
data['Genre'].value_counts()
data.Title = data.Title.apply(simple_preprocess, min_len=3)
data.Title.head()
stop_words = set(stopwords.words('english'))


def stemmingandstop(lis):
    lemmatizer = WordNetLemmatizer()
    filtered_lis = [lemmatizer.lemmatize(w) for w in lis if not w in stop_words and len(w) > 2]
    return filtered_lis


data.Title = data.Title.apply(stemmingandstop)
data.Title.head()
type(data.Title)
data.Title = data.Title.apply(' '.join)
data.head()
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
X = data.Title
y = data.Genre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_test)
print(accuracy_score(y_test, predictions))
svm_clftfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC()),
])
svm_clftfidf.fit(X_train, y_train)
tfsvmpred = svm_clftfidf.predict(X_test)
print(accuracy_score(y_test, tfsvmpred))
text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
])
parameters = {
 'vect__ngram_range': [(1, 1), (1, 2)],
 'tfidf__use_idf': (True, False),
 'clf__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
gs_value = gs_clf.predict(X_test)
print(accuracy_score(y_test, gs_value))
with open('/Users/rimshad/Downloads/scholar-vertical-search-engine-rimshad/scholar-vertical-search-engine-rimshad/Classification/imdb.csv') as f:
    dataj = json.load(f)
dataj = [row for row in dataj if not (row['Title'] is None)]
df = pd.DataFrame(dataj)
df.head()
classification = gs_clf.predict(df.title + df.description)
df['Genre'] = classification
df.to_json('classified.json', orient="records")


0.045
0.035


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


0.075


JSONDecodeError: Expecting value: line 1 column 1 (char 0)